#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [1]:
learning_rate = 1e-4
dropout_rate = 0.6
batch_size = 64
filters = [32, 64, 128, 256]
earlystop_patience = 14
reduce_lr_patience = 9
epochs = 40
target_size = (224, 224)

In [2]:
import os
import re
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                             accuracy_score, roc_auc_score, precision_score, recall_score)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, ReLU,
                                     MaxPooling2D, Dropout, Flatten, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#### 📁 **1. Görüntü Yolu Listeleme Fonksiyonu**


In [ ]:
#chatgpt
def mammo_gorunumlerini_listele(kok_dizin):
    pattern = re.compile(r'^[A-Z]_\d+_\d+\.(LEFT|RIGHT)_(CC|MLO)\.jpg$', re.IGNORECASE)
    siniflar = ['benign', 'cancer', 'normal']
    yollar, etiketler = [], []
    for cls in siniflar:
        cls_yolu = os.path.join(kok_dizin, cls)
        if not os.path.isdir(cls_yolu):
            continue
        for durum_id in os.listdir(cls_yolu):
            durum_yolu = os.path.join(cls_yolu, durum_id)
            if not os.path.isdir(durum_yolu):
                continue
            for dosya_adi in os.listdir(durum_yolu):
                if pattern.match(dosya_adi):
                    yollar.append(os.path.join(durum_yolu, dosya_adi))
                    etiketler.append(cls)
    return yollar, etiketler

#### 🧼 **2. Görüntü Ön İşleme**


In [4]:
def load_and_preprocess(paths, labels, target_size=target_size, method='minmax'):
    X, y = [], []
    label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
    for img_path, lbl in zip(paths, labels):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, target_size)
        img = img.astype(np.float32)
        if method == 'minmax':
            img /= 255.0
        else:
            img = (img - img.mean()) / (img.std() + 1e-8)
        X.append(img[..., np.newaxis])
        y.append(label_map[lbl])
    return np.array(X), np.array(y)

#### 🧠 **3. CNN Model Tanımı**

In [5]:
def _conv_block(inputs, filters):
    x = Conv2D(filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def create_cnn_model(input_shape=(224, 224, 1), num_classes=3, filters=filters, dropout_rate=dropout_rate):
    inp = Input(shape=input_shape)
    x = _conv_block(inp, filters[0])
    x = MaxPooling2D((2, 2))(x)

    x = _conv_block(x, filters[1])
    x = _conv_block(x, filters[1])
    x = MaxPooling2D((2, 2))(x)
    
    x = _conv_block(x, filters[2])
    x = _conv_block(x, filters[2])
    x = MaxPooling2D((2, 2))(x)
    
    x = _conv_block(x, filters[3])
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inp, outputs=out)

#### 🚂 **4. Model Eğitimi ve Kaydetme**


In [ ]:
if __name__ == '__main__':
    kok_dizin = os.path.join(os.getcwd(), "Project1")
    yollar, etiketler = mammo_gorunumlerini_listele(kok_dizin)
    X, y = load_and_preprocess(yollar, etiketler)

    X_egitim_val, X_test, y_egitim_val, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
    np.save("X_test_1_3.npy", X_test)
    np.save("y_test_1_3.npy", y_test)
    model = create_cnn_model()
    model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    geri_cagirmalar = [
        EarlyStopping(patience=earlystop_patience, restore_best_weights=True),
        ReduceLROnPlateau(patience=reduce_lr_patience, factor=0.5)
    ]
    #chatgpt
    model.fit(X_egitim_val, pd.get_dummies(y_egitim_val),
          epochs=epochs, batch_size=batch_size,validation_split=0.1,
          callbacks=geri_cagirmalar, verbose=1)

    olasiliklar = model.predict(X_test)
    esikler = np.linspace(0.4, 0.6, 21)
    en_iyi_f1, en_iyi_T_m, en_iyi_T_b = 0, 0.47, 0.53
    for T_m_aday in esikler:
        for T_b_aday in esikler:
            tahminler = [2 if p[2] >= T_m_aday else 1 if p[1] >= T_b_aday else 0 for p in olasiliklar]
            f1 = f1_score(y_test, tahminler, average='macro')
            if f1 > en_iyi_f1:
                en_iyi_f1, en_iyi_T_m, en_iyi_T_b = f1, T_m_aday, T_b_aday

Epoch 1/40
99/99 [==============================] - 27s 207ms/step - loss: 1.2546 - accuracy: 0.5101 - val_loss: 1.2286 - val_accuracy: 0.3201
Epoch 2/40
99/99 [==============================] - 18s 180ms/step - loss: 0.8362 - accuracy: 0.5481 - val_loss: 1.4891 - val_accuracy: 0.3201
Epoch 3/40
99/99 [==============================] - 18s 180ms/step - loss: 0.7846 - accuracy: 0.5966 - val_loss: 1.6807 - val_accuracy: 0.3201
Epoch 4/40
99/99 [==============================] - 18s 181ms/step - loss: 0.7584 - accuracy: 0.5854 - val_loss: 1.1337 - val_accuracy: 0.4538
Epoch 5/40
99/99 [==============================] - 18s 181ms/step - loss: 0.7346 - accuracy: 0.5946 - val_loss: 0.9118 - val_accuracy: 0.5448
Epoch 6/40
99/99 [==============================] - 18s 181ms/step - loss: 0.7269 - accuracy: 0.6059 - val_loss: 0.8921 - val_accuracy: 0.5676
Epoch 7/40
99/99 [==============================] - 18s 181ms/step - loss: 0.7124 - accuracy: 0.6074 - val_loss: 0.7800 - val_accuracy: 0.5932

 Metirkler**

In [7]:
nihai_tahminler = [2 if p[2] >= en_iyi_T_m else 1 if p[1] >= en_iyi_T_b else 0 for p in olasiliklar]
print(f"En iyi eşik sınırı: T_m={en_iyi_T_m:.2f}, T_b={en_iyi_T_b:.2f}")
print(f"en iyi f1 değeri: \nf1: {en_iyi_f1}")
print(classification_report(y_test, nihai_tahminler, target_names=['normal','benign','cancer']))
print(confusion_matrix(y_test, nihai_tahminler))
print("Accuracy:", accuracy_score(y_test, nihai_tahminler))
print("Precision:", precision_score(y_test, nihai_tahminler, average='macro'))
print("Recall:", recall_score(y_test, nihai_tahminler, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), olasiliklar, average='macro', multi_class='ovr'))
except:
    print("AUC hesaplanamadı (muhtemelen tek sınıf tahmini nedeniyle)")
with open("optimized_thresholds.txt", "w") as f:
    f.write(f"Best T_m: {en_iyi_T_m}\nBest T_b: {en_iyi_T_b}\nBest F1: {en_iyi_f1}")

En iyi eşik sınırı: T_m=0.41, T_b=0.52
en iyi f1 değeri: 
f1: 0.7331778505680333
              precision    recall  f1-score   support

      normal       0.84      0.86      0.85       241
      benign       0.75      0.56      0.64       268
      cancer       0.64      0.79      0.71       272

    accuracy                           0.73       781
   macro avg       0.75      0.74      0.73       781
weighted avg       0.74      0.73      0.73       781

[[207  14  20]
 [ 17 149 102]
 [ 21  35 216]]
Accuracy: 0.7323943661971831
Precision: 0.7454921553822653
Recall: 0.736336319379427
AUC: 0.8959789320257484


In [8]:
#### 💾 **6. Model ve Eşik Kayıtları**


In [9]:
model_name = "model3.h5"
model.save(model_name)
print(f"Model kaydedildi: {model_name}")

Model kaydedildi: model3.h5
